# lab 10. Moduł `array` oraz `datetime`

## 1. Moduł `array`

**_Listing 1_**

In [2]:
import sys
from array import array

# tablica zawierająca wartości typu unsigned int
# inicjalny rozmiar = 0 elementów
tab = array('I')
print(len(tab))

0


In [3]:
# możemy dodać element na koniec tablicy
tab.append(1)
print(len(tab))

1


In [4]:
# użycie metody extend
tab.extend([2, 3, 4])
print(tab)
print(len(tab))

# poniższy fragment zgłosi błąd TypeError
# tab.extend([2, 3.0, 4])
# print(tab)
# print(len(tab))

array('I', [1, 2, 3, 4])
4


In [5]:
# tablicę możemy również inicjować z istniejących obiektów
# z listy - wywołana zostanie metoda from list klasy array (patrz dokumentacja)
tab_from_list = array('I', [i for i in range(10)])
print(tab_from_list)

array('I', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [8]:
# z obiektu typu str - wywołana zostanie metoda fromunicode
tab_from_string = array('u', 'ABRACADABRA')
print(tab_from_string)
print(tab_from_string[4:6])
# wstawienie wartości z użyciem slice - typem obiektu wstawianego musi być również array
tab_from_string[4:6] = array('u', '_')
print(tab_from_string)

array('u', 'ABRACADABRA')
array('u', 'CA')
array('u', 'ABRA_DABRA')


C:\Users\Krzysztof\AppData\Local\Temp\ipykernel_54496\1948473521.py:2: DeprecationWarning: The 'u' type code is deprecated and will be removed in Python 3.16
  tab_from_string = array('u', 'ABRACADABRA')
C:\Users\Krzysztof\AppData\Local\Temp\ipykernel_54496\1948473521.py:6: DeprecationWarning: The 'u' type code is deprecated and will be removed in Python 3.16
  tab_from_string[4:6] = array('u', '_')


In [14]:
# z generatora
import random
tab_of_floats = array('f', (random.random() for _ in range(1000)))

# z ciekawości możemy porównać ilość zaalokowanej pamięci dla tablicy i listy tych samych wartości
# im więcej będzie wartości tym różnica będzie większa na korzyść tablicy
list_of_floats = [random.random() for _ in range(1000)]
print(sys.getsizeof(tab_of_floats))
print(sys.getsizeof(list_of_floats))

4200
8856


Dzięki tablicom możemy w środowisku z ograniczonymi zasobami zmniejszyć ilość zaalokowanej pamięci. A jak wygląda czas niezbędny do zaalokowania tablicy i listy tych samych elementów ?

**_Listing 2_**

In [29]:
# test czasu wykonania
from timeit import timeit
import random

setup = """
from array import array
import random

tab_of_floats = array('f', [random.random() for _ in range(1_000_000)])
"""

setup1 = """
import random

list_of_floats = [random.random() for _ in range(1_000_000)]
"""

stmt1 = """
sum(tab_of_floats)
"""
stmt2 = """
sum(list_of_floats)
"""

stmt3 = """
tab_of_floats[random.randint(0, len(tab_of_floats)-1)]
"""
stmt4 = """
list_of_floats[random.randint(0, len(list_of_floats)-1)]
"""


print(f"Suma elementów: {timeit(stmt1, setup, number=100)} vs. {timeit(stmt2, setup1, number=100)}")
print(f"Dostęp do elementów: {timeit(stmt3, setup, number=1000000)} vs. {timeit(stmt4, setup1, number=1000000)}")

Suma elementów: 0.8338521999994555 vs. 0.6286172000000079
Dostęp do elementów: 0.4928988000010577 vs. 0.6064598000011756


Widać, że czas wykonania operacji nie koniecznie jest zawsze po stronie tablic.

Pod [TYM](https://wiki.python.org/moin/TimeComplexity) linkiem znajduje się tablica złożoności obliczeniowej (w notacji `big O`) wybranych operacji w Pythonie.

**_Listing 3_**

In [31]:
# zapisanie tablicy do pliku oraz jej wczytanie
tab_of_floats = array('f', [random.random() for _ in range(1_000_000)])

with open('floats_array.bin', 'wb') as file_arr:
    tab_of_floats.tofile(file_arr)

# wczytujemy ponownie dane do tablicy floatów
tab_of_floats_loaded = array('f')
file_arr  = open('floats_array.bin', 'rb')
tab_of_floats_loaded.fromfile(file_arr, 1_000_000)
file_arr.close()
tab_of_floats[:10]

array('f', [0.7181819081306458, 0.027078989893198013, 0.7048152685165405, 0.6390222907066345, 0.8316575884819031, 0.18937312066555023, 0.15494900941848755, 0.7503846883773804, 0.49444857239723206, 0.2896447479724884])

In [12]:
# i analogiczna operacja dla listy
list_of_floats = [random.random() for _ in range(1_000_000)]
with open('floats_list.txt', 'w') as file_arr:
    file_arr.writelines('\n'.join([str(x) for x in list_of_floats]))

with open('floats_list.txt', 'r') as file_list:
    list_of_floats_loaded = file_list.readlines()

list_of_floats_loaded = [float(x.strip()) for x in list_of_floats_loaded]
print(list_of_floats_loaded[:10])

[0.12260510475958342, 0.8661683796035304, 0.10518445858826331, 0.8403406437764545, 0.7171574256819936, 0.6144930857488491, 0.9554982118657585, 0.970457856748045, 0.8796432542672188, 0.17391161595501003]


**_Listing 4_**

In [30]:
tab = array('I', range(1, 11))
print(tab)
# lokalizacja w pamięci
print(id(tab))

mem = memoryview(tab)
# lokalizacja w pamięci - inna, ale przecież to inny obiekt
print(id(mem))

# jednak elementy tablicy i memory view współdzielą pamięć
print(id(tab[0]))
print(id(mem[0]))

# nie wykonamy tego dla obiektów non byte
# print(mem.cast('I', [2, 5]))

tab = array('B', range(1, 11))
print(tab)
mem = memoryview(tab)
# jeżeli chcemy ten widok zmaterializować
print(mem.cast('B', [2, 5]).tolist())

array('I', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
1345155407264
1345155335552
140709833540520
140709833540520
array('B', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]


## 2. Moduł `datetime`

Moduł `datetime` oferuje funkcje pozwalające na manipulowanie obiektami daty i czasu (ale póki co nie datą i czasem ;-) ). Pozwala to z dużym ułatwieniem pracować z takimi wartościami bez konieczności ręcznej obsługi lat przestępnych, ilości dni w danym miesiącu czy stref czasowych.

> Dokumentacja:
> * https://docs.python.org/3/library/datetime.html

In [180]:
import datetime

# składowe modułu
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 'UTC',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'date',
 'datetime',
 'datetime_CAPI',
 'time',
 'timedelta',
 'timezone',
 'tzinfo']

Poniżej zostaną omówione wybrane funkcje tego modułu.

In [181]:
# zakres wartości składowej year w module datetime
datetime.MINYEAR, datetime.MAXYEAR

(1, 9999)

Wszystkie z wyżej wypisanych nazw (dir(datetime)) bez `__` są klasami w module datetime, i sa opisane tutaj: https://docs.python.org/3/library/datetime.html#available-types

Reprezentują obiekty daty (date), czasu (time), daty i czasu (datetime), strefy czasowej (tzinfo, timezone) oraz różnicy między dwoma datami (timedelta).

**Obiekt typu `date`**

Sygantura: `class datetime.date(year, month, day)`

In [184]:
d = datetime.date(2000, 1, 1)

In [185]:
d

datetime.date(2000, 1, 1)

In [187]:
d.year, d.month, d.day

(2000, 1, 1)

In [188]:
# metody statyczne
datetime.date.today()

datetime.date(2025, 5, 13)

In [190]:
# epoch start
datetime.date.fromtimestamp(0)

datetime.date(1970, 1, 1)

In [194]:
# 1000 dni później
# sekundy * minuty * godziny * dni
stamp = 60 * 60 * 24 * 1000
datetime.date.fromtimestamp(stamp)

datetime.date(1972, 9, 27)

In [195]:
# różnica między dwoma obiektami daty (daty i czasu) zwracana jest w postaci obiektu timedelta
# sprawdzamy
datetime.date.fromtimestamp(stamp) - datetime.date.fromtimestamp(0)

datetime.timedelta(days=1000)

In [197]:
# kolejna metoda statyczna tej klasy
datetime.date.fromisoformat('2023-01-12')

datetime.date(2023, 1, 12)

O formacie ISO 8601: https://en.wikipedia.org/wiki/ISO_8601

In [199]:
datetime.date.fromisoformat('20230112')

datetime.date(2023, 1, 12)

In [205]:
# inne funkcje tej klasy
data = datetime.date.today()

print(data.weekday())
print(data.isoweekday())
print(data.isocalendar())
print(data.isoformat())
print(data.ctime())

1
2
datetime.IsoCalendarDate(year=2025, week=20, weekday=2)
2025-05-13
Tue May 13 00:00:00 2025


**Obiekt typu `datetime`**

Sygnatura: class datetime.datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None, *, fold=0)

In [207]:
now = datetime.datetime.today()
now

datetime.datetime(2025, 5, 13, 12, 31, 43, 986635)

In [209]:
datetime.datetime.now()

datetime.datetime(2025, 5, 13, 12, 32, 39, 890781)

In [ ]:
# lista wszystkich dostępnych nazw stref czasowych
import pytz

print('Timezones')
for timeZone in pytz.all_timezones:
    print(timeZone)

In [227]:
from zoneinfo import ZoneInfo

# jaki aktualnie czas w Los Angeles?
datetime.datetime.now(ZoneInfo("America/Los_Angeles"))

datetime.datetime(2025, 5, 13, 3, 45, 19, 890703, tzinfo=zoneinfo.ZoneInfo(key='America/Los_Angeles'))

Kolejne bardzo przydatne funkcje związane z obsługą daty i czasu to możliwość konwersji z danego formatu.

```
datetime.strptime(date_string, format)
```

Formaty zapisu są dostępne w tabeli w dokumentacji modułu datetime pod adresem: https://docs.python.org/3/library/datetime.html#format-codes

In [230]:
datetime.datetime.strptime('2024/02/03', "%Y/%m/%d")

datetime.datetime(2024, 2, 3, 0, 0)

In [234]:
datetime.datetime.strptime('24/2/3', "%y/%m/%d")

datetime.datetime(2024, 2, 3, 0, 0)

In [237]:
start_of_this_year = datetime.datetime(2025, 1, 1)
start_of_this_year.timestamp()

1735686000.0

In [239]:
# możemy podmienić dowolna składową daty i czasu
start_of_this_year.replace(year=2020)

datetime.datetime(2020, 1, 1, 0, 0)

In [240]:
start_of_this_year.isoformat()

'2025-01-01T00:00:00'

## Zadania

**Zadanie 1**  
Wykorzystując przykład z listingu 2 napisz kod, który porówna czas inicjalizowania tablicy 10000 elementów z typem `i` oraz `w` (w tym drugim to łańcuchy znaków) i listy z takimi samymi wartościami. Porównaj również czas dostępu (jest już w przykładzie w listingu 2) oraz dodaj porównanie czasu wykonania 1000 operacji `insert` dla obu typów (tablica vs. lista).

**Zadanie 2**  
Zmierz czas operacji zapisu i ładowania danych z tablicy i listy z listingu 3. Użyj do tego modułu `datetime`. Wnioski ?


**Zadanie 3**  
Napisz funkcję, która przyjmuje datę urodzenia w postaci łańcucha znaków,a wyświetla na wyjściu komunikat:

Witaj! Na dzień dzisiejszy masz {tu wiek w latach} lat oraz {tu dodatkowo dni} dni. Razem daje to imponujące {tu dni od narodzin} dni!. Twoje najbliższe urodziny będą miały miejsce w dniu {tu data} czyli za {liczba miesięcy} miesięcy oraz {dodatkowo liczba dni} dni. Od poprzednich urodziny minęło {liczba miesięcy} miesięcy i {liczba dni} dni. 

**Zadanie 4**

W lab_02 znajduje się plik zamowienia.csv. Wczytaj ten plik (ale bez biblioteki pandas), a następnie zamień wszystkie wartości w kolumnie `Data zamowienia` na obiekt typu `datetime.date` i wyświetl:
* najstarszą datę,
* najnowszą datę,
* różnicę w dniach między najstarszą i najnowszą datą